In [11]:
import sys
import pathlib
import numpy as np
import pandas as pd
import optuna
import torch
import logging 

from optimize_utils import get_optimize_args, objective

script_directory = pathlib.Path("../utils/").resolve()
sys.path.insert(0, str(script_directory))
from data_loader import load_train_test_data

In [12]:
# Load command line arguments
args = get_optimize_args()


# Load data
data_directory = pathlib.Path("../0.data-download/data").resolve()

train_data, test_data, val_data, load_gene_stats = load_train_test_data(
    data_directory, train_or_test="all", load_gene_stats=True, zero_one_normalize=True
)


In [13]:
# Convert dataframes to tensors
train_tensor = torch.tensor(train_data, dtype=torch.float32)
test_tensor = torch.tensor(test_data, dtype=torch.float32)
val_tensor = torch.tensor(val_data, dtype=torch.float32)

In [14]:
# Run Optuna optimization and save study
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "BetaVAE-Optimization"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, direction="minimize", load_if_exists=True)
study.optimize(
    lambda trial: objective(trial, train_tensor, val_tensor, train_data), n_trials=500
)

[I 2024-07-23 10:41:38,137] Using an existing study with name 'BetaVAE-Optimization' instead of creating a new one.


Using an existing study with name 'BetaVAE-Optimization' instead of creating a new one.
Using an existing study with name 'BetaVAE-Optimization' instead of creating a new one.
Using an existing study with name 'BetaVAE-Optimization' instead of creating a new one.
Epoch 0, Loss: 231.88591126399254
Epoch 1, Loss: 208.90347772854477
Epoch 2, Loss: 197.54369024020522
Epoch 3, Loss: 185.84199947527986
Epoch 4, Loss: 176.1549994169776
Epoch 5, Loss: 164.62560780083956
Epoch 6, Loss: 156.12066960121268
Epoch 7, Loss: 146.87723151819029
Epoch 8, Loss: 136.25080457089553
Epoch 9, Loss: 127.65125932835821
Epoch 10, Loss: 117.64359841417911
Epoch 11, Loss: 115.74583430503732
Epoch 12, Loss: 115.36540490904851
Epoch 13, Loss: 106.95890275186566
Epoch 14, Loss: 98.32856663945896
Epoch 15, Loss: 102.32012593283582
Epoch 16, Loss: 94.26324845499067
Epoch 17, Loss: 96.20706696012127
Epoch 18, Loss: 94.18713269589553
Epoch 19, Loss: 91.83996691347947
Epoch 20, Loss: 90.16149501515858
Epoch 21, Loss: 84

[I 2024-07-23 10:41:47,340] Trial 191 finished with value: 46.58135477701823 and parameters: {'latent_dim': 58, 'beta': 1.161636019067799, 'learning_rate': 0.003200991469755715, 'batch_size': 98, 'epochs': 371}. Best is trial 191 with value: 46.58135477701823.


Epoch 369, Loss: 47.79626355527052
Epoch 370, Loss: 47.59808039878731
Trial 191 finished with value: 46.58135477701823 and parameters: {'latent_dim': 58, 'beta': 1.161636019067799, 'learning_rate': 0.003200991469755715, 'batch_size': 98, 'epochs': 371}. Best is trial 191 with value: 46.58135477701823.
Trial 191 finished with value: 46.58135477701823 and parameters: {'latent_dim': 58, 'beta': 1.161636019067799, 'learning_rate': 0.003200991469755715, 'batch_size': 98, 'epochs': 371}. Best is trial 191 with value: 46.58135477701823.
Trial 191 finished with value: 46.58135477701823 and parameters: {'latent_dim': 58, 'beta': 1.161636019067799, 'learning_rate': 0.003200991469755715, 'batch_size': 98, 'epochs': 371}. Best is trial 191 with value: 46.58135477701823.
Epoch 0, Loss: 240.22818038712686
Epoch 1, Loss: 219.35588561100747
Epoch 2, Loss: 202.85046787546642
Epoch 3, Loss: 189.91309322527985
Epoch 4, Loss: 178.7849026352612
Epoch 5, Loss: 169.57240409281715
Epoch 6, Loss: 159.695083663

[I 2024-07-23 10:41:56,369] Trial 192 finished with value: 47.688232421875 and parameters: {'latent_dim': 59, 'beta': 1.4733101773480652, 'learning_rate': 0.0031453039225855925, 'batch_size': 101, 'epochs': 364}. Best is trial 191 with value: 46.58135477701823.


...

Epoch 298, Loss: 48.29750757929104
Epoch 299, Loss: 48.239026061100745
Epoch 300, Loss: 48.24678681786381
Epoch 301, Loss: 48.17782547224814
Trial 204 finished with value: 48.00221252441406 and parameters: {'latent_dim': 61, 'beta': 2.3156803606151093, 'learning_rate': 0.0031318907104431666, 'batch_size': 86, 'epochs': 302}. Best is trial 191 with value: 46.58135477701823.
Trial 204 finished with value: 48.00221252441406 and parameters: {'latent_dim': 61, 'beta': 2.3156803606151093, 'learning_rate': 0.0031318907104431666, 'batch_size': 86, 'epochs': 302}. Best is trial 191 with value: 46.58135477701823.
Trial 204 finished with value: 48.00221252441406 and parameters: {'latent_dim': 61, 'beta': 2.3156803606151093, 'learning_rate': 0.0031318907104431666, 'batch_size': 86, 'epochs': 302}. Best is trial 191 with value: 46.58135477701823.
Epoch 0, Loss: 264.15438141324626
Epoch 1, Loss: 231.0907430620336
Epoch 2, Loss: 211.89611415578358
Epoch 3, Loss: 196.53717350746268
Epoch 4, Loss: 184.

[I 2024-07-23 10:43:58,625] Trial 205 finished with value: 48.206717597113716 and parameters: {'latent_dim': 61, 'beta': 2.362516127687036, 'learning_rate': 0.0031072942779993753, 'batch_size': 86, 'epochs': 310}. Best is trial 191 with value: 46.58135477701823.


Epoch 309, Loss: 48.151628818796645
Trial 205 finished with value: 48.206717597113716 and parameters: {'latent_dim': 61, 'beta': 2.362516127687036, 'learning_rate': 0.0031072942779993753, 'batch_size': 86, 'epochs': 310}. Best is trial 191 with value: 46.58135477701823.
Trial 205 finished with value: 48.206717597113716 and parameters: {'latent_dim': 61, 'beta': 2.362516127687036, 'learning_rate': 0.0031072942779993753, 'batch_size': 86, 'epochs': 310}. Best is trial 191 with value: 46.58135477701823.
Trial 205 finished with value: 48.206717597113716 and parameters: {'latent_dim': 61, 'beta': 2.362516127687036, 'learning_rate': 0.0031072942779993753, 'batch_size': 86, 'epochs': 310}. Best is trial 191 with value: 46.58135477701823.
Epoch 0, Loss: 263.2694525419776
Epoch 1, Loss: 230.068505130597
Epoch 2, Loss: 211.10730672807836
Epoch 3, Loss: 195.98000816231342
Epoch 4, Loss: 183.3031439482276
Epoch 5, Loss: 172.8887549556903
Epoch 6, Loss: 164.59963706856342
Epoch 7, Loss: 155.5359564

[I 2024-07-23 10:44:06,491] Trial 206 finished with value: 47.853915744357636 and parameters: {'latent_dim': 60, 'beta': 2.3467133849212587, 'learning_rate': 0.003070218408294996, 'batch_size': 85, 'epochs': 305}. Best is trial 191 with value: 46.58135477701823.


Epoch 302, Loss: 48.219988193796645
Epoch 303, Loss: 48.34801735949161
Epoch 304, Loss: 48.23827068271922
Trial 206 finished with value: 47.853915744357636 and parameters: {'latent_dim': 60, 'beta': 2.3467133849212587, 'learning_rate': 0.003070218408294996, 'batch_size': 85, 'epochs': 305}. Best is trial 191 with value: 46.58135477701823.
Trial 206 finished with value: 47.853915744357636 and parameters: {'latent_dim': 60, 'beta': 2.3467133849212587, 'learning_rate': 0.003070218408294996, 'batch_size': 85, 'epochs': 305}. Best is trial 191 with value: 46.58135477701823.
Trial 206 finished with value: 47.853915744357636 and parameters: {'latent_dim': 60, 'beta': 2.3467133849212587, 'learning_rate': 0.003070218408294996, 'batch_size': 85, 'epochs': 305}. Best is trial 191 with value: 46.58135477701823.
Epoch 0, Loss: 269.26143889925373
Epoch 1, Loss: 233.5311231926306
Epoch 2, Loss: 214.05987639925374
Epoch 3, Loss: 199.43566493703358
Epoch 4, Loss: 185.7766412080224
Epoch 5, Loss: 174.97

In [ ]:
# Save best hyperparameters
best_trial = study.best_trial
print(best_trial)
print(f"Best trial: {best_trial.values}")
print(f"Best hyperparameters: {best_trial.params}")

FrozenTrial(number=72, state=1, values=[102.11618381076389], datetime_start=datetime.datetime(2024, 7, 23, 10, 3, 22, 204507), datetime_complete=datetime.datetime(2024, 7, 23, 10, 3, 43, 382853), params={'latent_dim': 49, 'beta': 1.0018724495529896, 'learning_rate': 0.0027196821043995304, 'batch_size': 102, 'epochs': 966}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'latent_dim': IntDistribution(high=100, log=False, low=10, step=1), 'beta': FloatDistribution(high=10.0, log=False, low=1.0, step=None), 'learning_rate': FloatDistribution(high=0.005, log=False, low=1e-06, step=None), 'batch_size': IntDistribution(high=112, log=False, low=16, step=1), 'epochs': IntDistribution(high=1000, log=False, low=5, step=1)}, trial_id=73, value=None)
Best trial: [102.11618381076389]
Best hyperparameters: {'latent_dim': 49, 'beta': 1.0018724495529896, 'learning_rate': 0.0027196821043995304, 'batch_size': 102, 'epochs': 966}
